In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr
from scipy import stats
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import seaborn as sns
from datetime import datetime

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Install File

In [3]:
nameCSV = 'yasi.csv'
df = pd.read_csv(str(nameCSV), sep=r'\s+', engine='c', header=0, na_filter=False, \
                               dtype=np.float, low_memory=False)

data = df.sort_values(by=['tt'])

lat = data.values[:,0]
lon = data.values[:,1]
wh = data.values[:,2]
tt = data.values[:,3]
qc = data.values[:,4]
back =data.values[:,5]
ws = data.values[:,6]
len(tt)

3

In [6]:
alltime = num2date(tt[:],u'days since 1985-01-01 00:00:00 UTC')

In [57]:
for k in range(len(days)):
    print wh[days[k]], alltime[k]

 6.855 2011-02-01 23:38:12.579861


In [38]:
wh

array([6.855    , 7.329867 , 7.5814447])

In [39]:
lat

array([-16.520096, -17.490686, -18.2846  ])

In [58]:
lon

array([153.50105, 153.27458, 153.08806])

In [59]:
wh

array([6.855    , 7.329867 , 7.5814447])

In [41]:
num2date(tt[:],u'days since 1985-01-01 00:00:00 UTC')

array([datetime.datetime(2011, 2, 1, 23, 38, 12, 579861),
       datetime.datetime(2011, 2, 1, 23, 38, 28, 918749),
       datetime.datetime(2011, 2, 1, 23, 38, 42, 286458)], dtype=object)

In [55]:
days2

[datetime.datetime(2011, 2, 1, 23, 38, 12, 579861)]

In [ ]:
print wh

In [46]:
meanwh = np.mean(wh) #wave height for each satellite pass
meanwh


7.255437233333333

## WAVE PERIOD EQN

$ \epsilon = 3.25 \left( H_s^2 g^2/U^4 \right)^{0.31}$


In [10]:
def waveage(H, U, grav=9.80665):
    '''
    The pseudo wave age can be expressed in terms of significant wave height and surface wind speed.
    '''

    grav2 = grav**2
    hs2 = np.square(H) 
    u4 = np.power(U,4)
    tmp = np.divide(hs2*grav2,u4)
    eps = 3.25*np.power(tmp,0.31)
    
    return eps


def waveperiod(H, U, grav=9.80665):
    '''
    Wave age from GA-2, Govindan et al.
    '''

    eps = waveage(H, U, grav=9.80665)
    period = (((eps-(5.78))/(eps+(U/(H*((U/H)+H)))))+(H+(5.70)))
    
    return period

In [11]:
T = waveperiod(wh,ws)
print T
print len(T),len(wh)

[ 9.37746923 10.03578692 10.22269174]
3 3


In [49]:
#Get mean value to represent the whole satellite pass
meanwh = np.mean(wh)
meanT = np.mean(T)



## Calculate Wave Power



### Total Wave Energy
$E = \frac{1}{8} \left(pgH_s^2\right) $


In [13]:
def totalwaveenergy(H):
    
    '''
    The total wave energy can be calculated using Hs, gravity and water density
    '''
    
    x = 1./8.
    p = 1025. #sea water density, kg/m^3
    g = 9.80665
    h2 = np.square(H) 
    
    pgh = p*g*h2
    
    e = x*pgh
    
    return e

In [14]:
we = totalwaveenergy(wh)

In [15]:
we

array([59043.14358739, 67506.67893277, 72220.16844346])

### Wave energy speed / Wave Velocity
$Cg = \frac{gTz}{2\pi}n$


In [16]:
import math
def wavespeed(T):
    '''
    Wave group speed
    '''
    
    g =9.80665
    ts = T #wave period
    pi = 2*math.pi
    n=0.5
    
    top = g*ts
    
    Cg = n *(np.divide(top,pi))
    
    return Cg
    

In [17]:
Cg = wavespeed(T)
Cg

array([7.31806832, 7.83181181, 7.97767016])

### Wave Energy Flux
$P = ECg$

In [18]:
def waveenergyflux(H,T):
    '''
    The rate at which energy is carried by waves, determined using total wave energy and wave energy speed
    '''
    
    e = totalwaveenergy(H)
    Cg = wavespeed(T)
    y = 0.001
    P = e * Cg*y
    
    return P
    
    

In [19]:
power1 = waveenergyflux(wh,T)
power1

array([432.08175848, 528.69960527, 576.14868296])

In [50]:
pp = np.mean(power1)

In [52]:
pp

512.3100155701371

In [51]:
print pp.shape, power1.shape, 'therefore use pp in df'

() (3,) therefore use pp in df


In [22]:
print dayswh.shape
print daysTz.shape

(1,)
(1,)


## Max, min of wh

In [56]:
print 'min power', min(pp)
print 'max power', max(pp)

print 'min wh:', min(wh)
print 'max wh:', max(wh)

print 'min period:', min(T)
print 'max period:', max(T)

min power

TypeError: 'numpy.float64' object is not iterable